# Exploring Dimlp and Fidex rule generation for breast cancer classification

**Introduction:**

By the end of this notebook, you'll have a solid understanding of how to use Dimlp to train the model and the Fidex algorithms to extract rules.

**Objectives:**

    1. Understand the importance of interpretability in medical problems.
    2. Introduce Dimlp and Fidex as powerful XAI techniques.
    3. Understand how to use Dimlp and Fidex.
    4. Showcase the capabilities of HES-Xplain in implementing Dimlp and Fidex algorithms.
    5. Provide practical insights into applying Dimlp and Fidex to breast cancer classifiers through an interactive notebook.
    6. Foster a community of XAI enthusiasts and practitioners.

**Outline:**

    1. Dataset and Problem Statement.
    2. Load the dataset.
    3. Model training.
    4. Local rules generation - Fidex.
    5. Global ruleSet generation - FidexGlo.
    6. Conclusion.
    7. References.

# Dataset and Problem Statement

The Breast Cancer Dataset used in this use case consists of 569 data samples representing patients who have tumors. Each patient has 30 attributes, computed out of an image, used to determine if the tumor is malignant (first class) or benign (second class). This dataset is available on [Kaggle](https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data) and can be imported with sklearn. It is composed of 455 training samples and 114 testing samples. 

The 30 attributes are composed of 10 real-valued features computed for each cell nucleus :

- **Radius**: Mean of distances from the center to points on the perimeter.
- **Texture**: Standard deviation of gray-scale values.
- **Perimeter**
- **Area**
- **Smoothness**: Local variation in radius lengths.
- **Compactness**: $\frac{\text{perimeter}^2}{\text{area}} - 1.0$
- **Concavity**: Severity of concave portions of the contour.
- **Concave Points**: Number of concave portions of the contour.
- **Symmetry**
- **Fractal Dimension**: "Coastline approximation" - 1.

For each tumor image, the mean, standard error, and worst errors were computed and appear as different attributes.

**Problem Statement:** Our objective is to build a robust classifier capable of accurately classifying breast cancer samples among the 2 classes. By leveraging deep learning techniques and Fidex algorithms, we aim to not only achieve high classification performance but also gain insights into the attributes that contribute to the classification decisions.

# Load the dataset

We'll start by importing all libraries.

In [1]:
import os, sys
from dimlpfidex.fidex import fidex, fidexGloRules, fidexGloStats, fidexGlo
from trainings.cnnTrn import cnnTrn

#cnnTrn('--json_config_file data/MnistTemplates/config_cnn.json')   

/home/jeanmarc/dimlpfidex/notebooks/.venv/lib/python3.10/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


Parameters list:
 - json_config_file                                              data/MnistTemplates/config_cnn.json
 - root_folder                                                   data/MnistDataset
 - train_data_file                                               data/MnistDataset/trainData.txt
 - train_class_file                                              data/MnistDataset/trainClass.txt
 - test_data_file                                                data/MnistDataset/testData.txt
 - test_class_file                                               data/MnistDataset/testClass.txt
 - original_input_size                                           (28, 28)
 - nb_channels                                                   1
 - model                                                         small
 - data_format                                                   classic
 - nb_classes                                                    10
 - train_valid_pred_outfile                               

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 24, 24, 32)     │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d                │ (None, 8, 8, 32)       │           832 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 4, 4, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,562 (529.54 KB)

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/80

Epoch 1: val_loss improved from inf to 0.00469, saving model to data/MnistDataset/weightsModel.keras


2024-06-25 16:03:16.286324: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-25 16:03:22.505217: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1688/1688 - 27s - 16ms/step - accuracy: 0.8913 - loss: 0.0175 - val_accuracy: 0.9783 - val_loss: 0.0047
Epoch 2/80

Epoch 2: val_loss improved from 0.00469 to 0.00291, saving model to data/MnistDataset/weightsModel.keras
1688/1688 - 14s - 8ms/step - accuracy: 0.9735 - loss: 0.0049 - val_accuracy: 0.9857 - val_loss: 0.0029
Epoch 3/80

Epoch 3: val_loss improved from 0.00291 to 0.00249, saving model to data/MnistDataset/weightsModel.keras
1688/1688 - 14s - 8ms/step - accuracy: 0.9795 - loss: 0.0036 - val_accuracy: 0.9875 - val_loss: 0.0025
Epoch 4/80

Epoch 4: val_loss improved from 0.00249 to 0.00217, saving model to data/MnistDataset/weightsModel.keras
1688/1688 - 14s - 8ms/step - accuracy: 0.9835 - loss: 0.0030 - val_accuracy: 0.9872 - val_loss: 0.0022
Epoch 5/80

Epoch 5: val_loss improved from 0.00217 to 0.00170, saving model to data/MnistDataset/weightsModel.keras
1688/1688 - 14s - 8ms/step - accuracy: 0.9856 - loss: 0.0026 - val_accuracy: 0.9903 - val_loss: 0.0017
Epoch 6/80


KeyboardInterrupt: 

We download the dataset with sklearn, then shuffle it randomly.

In [ ]:
fidexGloRules('--json_config_file data/MnistTemplates/config_fidexGloRules.json')  

In [ ]:
# datas, classes = shuffle(load_breast_cancer().data, load_breast_cancer().target)

dataset, classes = load_breast_cancer(return_X_y=True, as_frame=True)
classesLabels = list(load_breast_cancer().target_names)

# classes one-hot encoding 
classes = pd.get_dummies(classes, dtype="uint8")

# rename classes
classes = classes.rename(
    columns={
        classes.columns[0]: classesLabels[0],
        classes.columns[1]: classesLabels[1],
    }
)

dataset.head()

We separate the dataset into a training set (75%) and test set (25%).

In [ ]:
nbSamples = dataset.shape[0]
nbAttributes = len(dataset.columns)
nbClasses = len(classes.columns)

cutoff = int(0.75 * nbSamples)

trainData = dataset.iloc[:cutoff, :]
testData = dataset.iloc[cutoff:, :]
trainClasses = classes.iloc[:cutoff]
testClasses = classes.iloc[cutoff:]

assert (trainData.shape[0] + testData.shape[0]) == nbSamples
assert (trainClasses.shape[0] + testClasses.shape[0]) == nbSamples

We display the number of training and testing data and check that there is the same number of data as classes.

In [ ]:
print(f"There are {trainData.shape[0]} train datas and {trainClasses.shape[0]} train classes")
print(f"There are {testData.shape[0]} test datas and {testClasses.shape[0]} test classes")

Now, we save the all the data in text files in a specific format into BCWDataset directory. 

In [ ]:
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")

rootDir = "data/BCWDataset/"
create_folder(rootDir)

# defining filenames
datasetFile = "datas.txt"
trainDataFile = "trainData.txt"
testDataFile = "testData.txt"
trainClassFile = "trainClass.txt"
testClassFile = "testClass.txt"

dataset.to_csv(rootDir+datasetFile, header=False, index=False)
3
testData.to_csv(rootDir+testDataFile, header=False, index=False)

Finally, we normalize the data so that the model learns better. We save the mean and std of each attribute in the file normalization_stats.txt. We use the normalization file located in the trainings module. To display the parameters, you can just call normalization with no parameters, or with -h or --help. Without further details, we normalize with this command :

In [ ]:
#normalization("--help")

args = f"""
        --data_files [{trainDataFile},{testDataFile}] 
        --nb_attributes {nbAttributes} 
        --missing_values NaN 
        --root_folder {rootDir}
        """
normalization(args)

To show attribute names:

In [ ]:
attributes = list(dataset.columns)
print(attributes)

And class names:

In [ ]:
classes = list(classes.columns)
print(classes)

We store attribute and class names in a file named `attributes.txt` to identify features within rules.

In [ ]:
attributesFileName = "attributes.txt" 
attributesFileContent = attributes + classesLabels
attributesFileContent = [x.replace(" ", "_") for x in attributesFileContent]

with open(rootDir+attributesFileName, "w") as f:
    for item in attributesFileContent:
        f.write(item+'\n')

# Model training
Here is an example of how to train a DimlpBT model with our dataset.

First, let's display all the DimlpBT parameters available:

To configure our training, we will use this configuration file, located at `data/BCWTemplates/config_dimlpBT.json`:

```json
{
    "root_folder": "data/BCWDataset",
    "train_data_file": "trainData_normalized.txt",
    "train_class_file": "trainClass.txt",
    "test_data_file": "testData_normalized.txt",
    "test_class_file": "testClass.txt",
    "attributes_file": "attributes.txt",
    "train_pred_outfile": "predTrain.out",
    "test_pred_outfile": "predTest.out",
    "console_file": "resultDimlpBT.txt",
    "with_rule_extraction": true,
    "global_rules_outfile": "dimlpBTRules.rls",
    "weights_outfile": "weights.wts",
    "stats_file": "stats.txt",
    "nb_attributes": 30,
    "nb_classes": 2,
    "hidden_layers": [5]
}
```

> A detailed list of parameters is available [here](https://hes-xplain.github.io/documentation/algorithms/dimlp/dimlpBT/). If you want to create your own configuration file, we recommend using our [configuration file creator](https://hes-xplain.github.io/documentation/gui/)



We can start training our model: 

In [ ]:
result = dimlpBT("--json_config_file data/BCWTemplates/config_dimlpBT.json")

You can check all outputs inside the `data/BCWDataset` folder.

Inside `stats.txt` you can see the train and test accuracy as well as the sum squared errors. 

There are also the train predictions, test predictions, the weights, and the dimlp rules files.

In the file `dimlpBTRules.txt`, you can see the rule set, some statistics on the rules, and some statistics on the ruleset. Each rule is composed of antecedents and target class as well as the number of covering of the rule, which is the number of examples that verify (or "activate") the rule (even if the class is not correct).

In the `Training set` and `Testing set` sections, each rule is composed, left to right, of:

The number of covered samples, the number of correct covered samples, the number of false covered samples, and the accuracy of the rule.

Between the `Training set` and `Testing set` sections, There are statistics of the ruleset:

- The number of rules in the set.
- The mean total and the mean number of antecedents per rule.
- The covering of the rule.
- The rules' accuracy.
- The fidelity to the model is the percentage of covered samples that are correct concerning the model's decision.
- The model accuracy if we keep only the samples for which the model and the rules decision agree.
- The default rule activation rate, which is the percentage of samples for which no rule is activated.

# Local rules generation - Fidex

Fidex is an algorithm used in classification problems that allows us to obtain a rule explaining the decision class of the model for a given test sample. It enables us to grasp the model's decision-making process and to better understand the importance of each parameter in discerning the nature of the tumor, distinguishing between benign and malignant cases.

Now we can generate some local rules to explain the models' results. We can start with launching Fidex on one test sample. This will generate a rule explaining the sample locally. It is local because the algorithm searches a rule only for one sample.
Fidex is located in the fidex module. Let's take a look at the parameters :

In [ ]:
res = fidex("--help")

We see that we need the number of attributes and classes, the train and test data and class files, as well as train and test predictions, weights file name, and rule output file name where the rules will be saved.<br>
We specify as well the saved folder and the attributes.<br>
We also need to specify the normalization file obtained from training, to denormalize the values in the generated rule, otherwise the values will be normalized and impossible to interpret.<br>

To see what happens, we launch it with just one sample, and we save beforehand the test data sample in a file with its class and predictions:

In [ ]:
testPredsFile = "predTest.out"
testSampleFile = "testDataSample.txt"
testPreds = pd.read_csv(rootDir+testPredsFile, sep=" ", header=None, index_col=None).iloc[:, :nbClasses]
testDatasNormalized = pd.read_csv(rootDir+testDataFile, header=None, index_col=None)

sampleSelected = 0
assert(sampleSelected < nbSamples)

# extract a sample to generate local rule
sampleData = testData.iloc[sampleSelected, :nbAttributes].to_list()
samplePred = testPreds.iloc[sampleSelected, :].to_list()
sampleClasses = testData.iloc[sampleSelected, nbAttributes:].to_list()

# write the sample, classes and predictions in the testSampleFile file (file writing format must be respected)
with open(rootDir+testSampleFile, 'w') as f:
    f.write(" ".join(str(x) for x in sampleData) + '\n')
    f.write(" ".join(str(x) for x in samplePred) + '\n')
    f.write(" ".join(str(x) for x in sampleClasses) + '\n')

#res = fidex.fidex("--train_data_file trainData_normalized.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --test_data_file testDataSample.txt --weights_file Weights/weights.wts --attributes_file attributes.txt --rules_outfile rule.rls --nb_attributes 30 --nb_classes 2 --normalization_file normalization_stats.txt --root_folder data/BCWDataset")
res = fidex("--json_config_file data/BCWTemplates/config_fidexOne.json")

You can see the walkthrough of the algorithm and the rule extracted. The rule is also saved in the rule.rls file. With the rule, we see also the covering size of the rule on the training set, the fidelity, the accuracy, and the confidence of the rule. The confidence shows how much the rule is confident with his choices, with respect to the prediction values.<br>

Now, we execute Fidex with all test samples. We send the console output in the fidexResult.txt file and save the global statistics in fidexStats.txt. **It should take about a minute**.

In [ ]:
#res = fidex.fidex("--train_data_file trainData_normalized.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --test_data_file testData_normalized.txt --test_class_file testClass.txt --test_pred_file predTest.out --weights_file Weights/weights.wts --attributes_file attributes.txt --rules_outfile fidexRules.rls --stats_file fidexStats.txt --console_file fidexResult.txt --nb_attributes 30 --nb_classes 2 --normalization_file normalization_stats.txt --root_folder data/BCWDataset")
res = fidex("--json_config_file data/BCWTemplates/config_fidex.json")

if (res == 0):
    print("Fidex done")

You can see the rules generated for each sample in the file fidexRules.rls. The global statistics on the test set appear in statsFidex.txt. There is the mean covering size per rule, the mean number of antecedents per rule, and the mean rule fidelity, accuracy, and confidence.

# Global ruleSet generation - FidexGlo
We've seen how to compute a rule that explains the decision of the model for a specific sample. Now, we will generalize a ruleset that characterizes the whole train dataset. That means that for each training sample, there is a rule in the set of rules that explains the model's decision for this sample. We will use this global ruleset to explain the results obtained on new test samples. If there is a rule of the ruleset corresponding to the sample, we take this one and get a global explanation for the sample. If there is none, we call Fidex and only have a local explanation.<br>

To get the ruleSet we execute fidexGloRules which is located in the fidex module. Here are the possible parameters :

In [ ]:
res = fidexGloRules("--help")

We use nearly the same parameters as for Fidex but we only need train data. We need to choose a heuristic for fidexGloRules, we choose the optimal to get better results. We don't forget to add the normalization file.<br>
**It should take about 3 minutes**. If you have several processors available, you should add the parameter nb_threads with the number of processors that you want to use, it can speed up the process a lot. If you want to accelerate the process even more, you can use some dropout, the algorithm will randomly skip some dimensions or some hyperplans. For example, you can put: -d 0.5 -h 0.5, to skip half dimensions and half hyperplans, which should be about 4 times faster.

In [ ]:
#res = fidex.fidexGloRules("--train_data_file trainData_normalized.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --weights_file Weights/weights.wts --attributes_file attributes.txt --nb_attributes 30 --nb_classes 2 --heuristic 1 --global_rules_outfile globalRules.rls --console_file fidexGloRulesResult.txt --normalization_file normalization_stats.txt --root_folder data/BCWDataset")
res = fidexGloRules("--json_config_file data/BCWTemplates/config_fidexGloRules.json")
if (res == 0):
    print("FidexGloRules done")

You can see the console result in the file fidexGloRulesResult.txt and the ruleset in the file globalRules.rls.
The algorithm is random, so each execution may generate different rules and a different number of them. It should generate about 25 rules. You can see at the top of the globalRules file the number of rules, the mean covering number per rule, and the mean number of antecedents. Here is an example of a rule that you may obtain:<br>

Rule 1: WORST_CONCAVE_POINTS<0.111377 AREA_ERROR<37.691316 -> BENIGN <br>
   Train Covering size : 217 <br>
   Train Fidelity : 1 <br>
   Train Accuracy : 0.990783 <br>
   Train Confidence : 0.984811 <br>

This rule is the first rule, which means that it's the rule with the maximum covering. Here, 217 train samples verify this rule. She is 100% fidel with the model and has about 99% train accuracy.
This rule says that if you have worst concave points lower than 0.111377 and an area error lower than 37.691316, then your breast cancer is benign. And this rule is accurate, on train test, at 99% and has 98% of confidence.<br>

Now, we can see some global statistics on the test set, and also some statistics directly on each rule. So we will see the test accuracy on this rule.<br>
We execute fidexGloStats which is located in the fidex module. First, let's check the parameters :

In [ ]:
res = fidexGloStats("--help")

We need the test files(data, prediction, class), the rules file, and the number of attributes and classes. We need to specify the attribute file as well. We choose to save the results in the file fidexGloStats.txt. With --global_rules_outfile we can generate the statistics on rules which will modify the rules file. If you want to keep the first ruleSet unchanged, you should give another name. Finally, with --positive_class_index we can specify the positive class in order to get a ROC curve and some other statistics on false and true positives/negatives. We will design the malignant class as positive. So here is the command :

In [ ]:
#res = fidex.fidexGloStats("--test_data_file testData.txt --test_pred_file predTest.out --test_class_file testClass.txt --attributes_file attributes.txt --global_rules_file globalRules.rls --global_rules_outfile globalRules.rls --nb_attributes 30 --nb_classes 2 --stats_file fidexGloStats.txt --positive_class_index 0 --root_folder data/BCWDataset")
res = fidexGloStats("--json_config_file data/BCWTemplates/config_fidexGloStats.json")

Here you can see the global statistics on the test set. As the previous calculations have randomness, each execution is a bit different. So here we will discuss the same case as before, so you may have slightly different results. In our case, we had about 98% fidelity, which is good, and a rule accuracy(96.5%) about 1.5% lower than the model accuracy(98%). So the rules seem to classify a bit worse. The explainability rate is the percentage of samples for which we can find a rule in the rules set. For the others, we need to execute Fidex (this is the default rule rate). In our case, we had more than a 97% explainability rate, so only in 3% of cases do we need to compute Fidex. Each rule can activate many rules. Here on average, a sample activates 5 correct rules and 0.05 wrong rules. A wrong rule is a rule with which the model doesn't agree. For example, if the rule says malign and the model says benign. Something interesting is the model test accuracy when rules and model agree. You can see that, generally, the accuracy increases if we consider samples where rules and model agree, and increases even more if we take only the activated rules (when there are no activated rules, we choose the model prediction). That means that the rules confirm well the model decision, but when no rule is found, the model decision may as well be wrong. <br>

Finally, we have the statistics on the positive/negative decisions of the model and of the rules. For the model's decisions, we had 1% false positives and 7% false negatives. That means that when the model says that it's malign, it has a 99% chance of being malign. But when it says that it's benign, there is a 7% risk that it is malign in reality. So the precision is 97% and the recall is 93%.

In the case of the rules decisions, the rules decision is the same as the model if we find a correct rule. If there is no rule, we can launch Fidex, so it's also the same decision as the model because it will create a fidel rule. The only scenario when the decision changes from the model's is when some rules are activated, all rules decide the same class which is not the class chosen by the model. The results are really close with 95% of precision and 93% of recall.

In globalRules file, you can now see the statistics of rules on the test set. Here is the same rule as before that I have now:<br>

Rule 1: MEAN_CONCAVE_POINTS<0.041282 WORST_CONCAVE_POINTS<0.150829 -> BENIGN <br>
   Train Covering size : 217 --- Test Covering size : 54 <br>
   Train Fidelity : 1 --- Test Fidelity : 0.981481 <br>
   Train Accuracy : 0.990783 --- Test Accuracy : 1 <br>
   Train Confidence : 0.984811 --- Test Confidence : 0.979816 <br>

You can see that the rule no longer always agrees with the model, only in 98.15% of cases. However, the rule accuracy has increased with the test. That means that the rule is very good in reality, with 100% of correct classification.

We can get a ROC curve obtained with the test set. Here are the parameters of computeRocCurve which is located in the trainings package :

In [ ]:
res = computeRocCurve("--help")

We execute computeRocCurve :

In [ ]:
#res = computeRocCurve('--test_class_file testClass.txt --test_pred_file predTest.out --positive_class_index 0 --output_roc outRoc.png --stats_file stats.txt --root_folder data/BCWDataset --nb_classes 2 --show_params false')
res = computeRocCurve("--json_config_file data/BCWTemplates/config_roc.json")

The AUC score is added to the stats file and the ROC curve is saved in the outRoc.png file. We get an AUC of about 99%. <br>
We can visualize the AUC curve :

In [ ]:
display(Image(filename="data/BCWDataset/outRoc.png"))

Finally, it's possible to launch fidexGlo on a test set. It will, for each test sample, search in the global ruleset for all the rules that verify the sample's properties and match the decision of the model. If no rule verifies the properties, it calls Fidex on the test sample. In the other case, it gives all the correct and wrong activated rules.<br>

FidexGlo is located in the fidex module. Here are the possible parameters:

In [ ]:
res = fidexGlo("--help")

As we want to use Fidex on test sample not covered by the global ruleset, we need to specify the Fidex parameters. We don't forget to add the normalization file and launch it with :

In [ ]:
#res = fidex.fidexGlo("--test_data_file testData.txt --test_pred_file predTest.out --global_rules_file globalRules.rls --nb_attributes 30 --nb_classes 2 --attributes_file attributes.txt --explanation_file explanations.txt --console_file fidexGloResults.txt --root_folder data/BCWDataset --with_fidex true --train_data_file trainData.txt --train_pred_file predTrain.out --train_class_file trainClass.txt --test_class_file testClass.txt --weights_file Weights/weights.wts --normalization_file normalization_stats.txt")
res = fidexGlo("--json_config_file data/BCWTemplates/config_fidexGlo.json")

if res == 0:
    print("Explanations generated") 

The explanations for each test sample can then be found in the file explanations.txt. We do not find a global rule for around 2.6% of the samples, for which we calculate a local rule using Fidex.

# Conclusion

Throughout this notebook, we've learned how to use DimlpBT to train a dataset and get some explaining rules, and we got into the different Fidex algorithms to get some local and global explanations, and some statistics on the model's decisions. We used the example of the breast cancer dataset from Wisconsin and we saw that the algorithms perform very well. It gives some good explanations of the model's decision and does it very fast, especially if a fidexGlo ruleset has been computed beforehand. You can try with any tabular dataset you want, you just have to remember to transform the data in a good format. <be>

We didn't speak about every single parameter, you can try to change every hyperparameter you want to see how it goes. An interesting parameter that we didn't consider is the parameter for decision threshold (--decision_threshold) in Fidex algorithms. It allows us to change prediction with respect to a specific threshold on the positive class. If the model gives a score prediction, for the positive class, greater than this threshold, the model prediction is considered to be the positive class, even if another class obtains a higher score. If the recall is not good enough, it's possible to improve it this way. Another parameter is the seed. It allows you to remove the randomness of the algorithms and get the same result in each execution for the same parameters and data.<br>

We considered only a simple tabular dataset. It's also possible to use it for an image classification problem. To see how we can train a dataset with convolutions, we recommend exploring the Mnist notebook for a hands-on experience **Insérer lien notebook Mnist**. There are also other training methods, like an MLP or decision trees. The Keel Australian notebook goes through these and shows what to do when we have categorical attributes **Insérer Keel Australian notebook**.

# Références

Article DimlpBT de Guido
Article Fidex de Guido (à venir)